# OPTIMIZING GRADIENT DESCENT

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
#pip install tensorflow-addons

In [3]:
import pandas as pd
import datetime, os
import numpy as np
import numpy.random as npr
from pylab import plt, mpl
import time

from scipy.stats import norm
from scipy import optimize
import scipy.integrate as integrate
import scipy.special as special 

import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorboard.plugins.hparams import api as hp
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

import tensorflow_addons as tfa

import matplotlib.pyplot as plt
import seaborn as sns

from keras import backend as BK

# Load the TensorBoard notebook extension
%load_ext tensorboard

Using TensorFlow backend.


Lets read the CSV so we dont need to run it again, but can just load it when needed.

In [4]:
#To read the import the csv-file, use:
raw_Options_input = pd.read_csv (r"/Users/Marcklein/Desktop/Master Thesis/Option pricing using Neural Networks/Python/Heston/Options_input.csv")
raw_Options_output = pd.read_csv (r"/Users/Marcklein/Desktop/Master Thesis/Option pricing using Neural Networks/Python/Heston/Options_output.csv")

#Creates some unnamed column in the beginning, delete it:
del raw_Options_input['Unnamed: 0']
del raw_Options_output['Unnamed: 0']


We train on the first 5k input/output parameters to speed up the process

In [5]:
raw_Options_inputz = raw_Options_input[:5000]
raw_Options_outputz = raw_Options_output[:5000]

In [6]:
Options_input = raw_Options_input.copy()
Options_output = raw_Options_output.copy()

Since the standard deviation is calculated by taking the sum of the squared deviations from the mean, a zero standard deviation can only be possible when all the values of a variable are the same (all equal to the mean). In this case, those variables have no discriminative power so they can be removed from the analysis. They cannot improve any classification, clustering or regression task. Many implementations will do it for you or throw an error about a matrix calculation.

### **Data preparation**

We split our dataset into a training set and a test set (validation set is taken from the training set during model.fit).

In [7]:
# 90% for training and validating
train_dataset = Options_input.sample(frac=0.9, random_state=42)
test_dataset = Options_input.drop(train_dataset.index)

train_labels = Options_output.sample(frac=0.9, random_state=42)
test_labels = Options_output.drop(train_labels.index)

Check the overall statistics

In [8]:
train_stats = train_dataset.describe().T

In [9]:
#normalize the data
def norm(x):
    return (x - train_stats['mean']) / train_stats['std']
normed_train_data = norm(train_dataset).values
normed_test_data = norm(test_dataset).values

#make the labels into numpy array just like the normed training data
train_labels = np.asarray(train_labels)
test_labels = np.asarray(test_labels)

#check the shapes
print("Input train data:", normed_train_data.shape, " Output train data:", train_labels.shape)
print("Input test data:", normed_test_data.shape, " Output test data:", test_labels.shape)

Input train data: (90000, 7)  Output train data: (90000, 10)
Input test data: (10000, 7)  Output test data: (10000, 10)


## **Optimizing gradient descent**

### Definitions
EPOCH: A full training pass over the entire dataset such that each example has been seen once. Thus, an epoch represents N/batch size training iterations, where N is the total number of examples.

ITERATION: A single update of a model's weights during training. An iteration consists of computing the gradients of the parameters with respect to the loss on a single batch of data (length of input data / batch_size = iterations).

BATCH: The set of examples used in one iteration (that is, one gradient update) of model training.

BATCH SIZE: The number of examples in a batch. For example, the batch size of SGD is 1, while the batch size of a mini-batch is usually between 10 and 1000. Batch size is usually fixed during training and inference; however, TensorFlow does permit dynamic batch sizes.

STEP: A forward and backward evaluation of one batch

STEP SIZE = LEARNING RATE: A scalar used to train a model via gradient descent. During each iteration, the gradient descent algorithm multiplies the learning rate by the gradient. The resulting product is called the gradient step.

- https://medium.com/@ewuramaminka/epoch-iterations-batch-size-11fbbd4f0771
_____________________________________________________________________________

Plotting examples of Batch gradient descent, Stochastic gradient descent and mini-batch stochastic gradient descent for the Heston model.

NB: For the SGD: increase increase the learning rate to 0.5 in order to get a better illustration (even though it may be a bit of an extreme) and send the update_frequency to 'batch'. Also set the timer to 5 minutes.

NB: For the BSG/Mini-SGD image 1: set the timer to 5 minutes and batch_size for Mini-SGD to 256 (i.e. 4500/256=18 iterations per epoch for Mini-SGD and 4500/4500=1 iteration for BSG). epochs can just be set to n_epochs to ensure that we do not stop before the 5 minutes. Look at the batch loss

NB: For the BSG/Mini-SGD image 2: comment out the timer and set the number of epochs to 5000. Look at the epoch loss

In [10]:
# Clear any logs from previous runs
!rm -rf ./logs/

In [11]:
'''# input_size = len(train_dataset.keys())
output_size = 10
hidden_layer_size = 100
n_epochs = 100000

weights_initializer = keras.initializers.GlorotUniform()
#weights_initializer = tf.random_normal_initializer(mean=0.0, stddev=0.05, seed=None)

#A function that trains and validates the model and returns the MSE (for the purpose of time-constrained comparison of BSG and Mini-SGD)
def train_val_model(run_dir, hparams):
    model = keras.models.Sequential([
            #Layer to be used as an entry point into a Network
            keras.layers.InputLayer(input_shape=[len(train_dataset.keys())]),
            #Dense layer 1
            keras.layers.Dense(hidden_layer_size, activation='sigmoid', 
                               kernel_initializer = weights_initializer,
                               name='Layer_1'),
        
            #Batch Layer
            keras.layers.BatchNormalization(),
        
            #Dense layer 2
            keras.layers.Dense(hidden_layer_size, activation='sigmoid', 
                               kernel_initializer = weights_initializer,
                               name='Layer_2'),
        
            #Batch Layer
            keras.layers.BatchNormalization(),
        
            #activation function is linear since we are doing regression
            keras.layers.Dense(output_size, activation='linear', name='Output_layer')
                                ])
    
    #Use the stochastic gradient descent optimizer but change batch_size to get BSG, SGD or MiniSGD
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.5, momentum=0.0,
                                        nesterov=False)
    
    #Compiling the model
    model.compile(optimizer=optimizer, 
                  loss='mean_squared_error', #Computes the mean of squares of errors between labels and predictions
                  metrics=['mean_squared_error']) #Computes the mean squared error between y_true and y_pred
    
    # initialize TimeStopping callback 
    time_stopping_callback = tfa.callbacks.TimeStopping(seconds=5*60, verbose=1)
    
    #Training the network
    history = model.fit(normed_train_data, train_labels, 
         epochs=hparams['epochs'],
         batch_size=hparams['batch_size'], 
         verbose=2,
         #validation_split=0.2,
         callbacks=[tf.keras.callbacks.TensorBoard(run_dir + "/Heston", update_freq='batch'),
                    time_stopping_callback])
    
    return history

train_val_model("logs/sample", {'batch_size': len(normed_train_data), 'epochs': 50000})
train_val_model("logs/sample1", {'batch_size': 256, 'epochs': 50000})''';

In [12]:
#%tensorboard --logdir_spec=BSG:logs/sample,Mini_SGD:logs/sample1
#%tensorboard --logdir logs/sample1

Computing the learning rates instead. One with a constant learning rate of 0.001 and another starting from 0.1 and decreasing exponentially. Remember to set the dataset back to its full length

In [13]:
learning_rate_fn = tf.keras.optimizers.schedules.PolynomialDecay(initial_learning_rate = 0.2,
                                                  decay_steps = 1000, end_learning_rate=0.001, power=1.0, cycle=False, name=None)

# input_size = len(train_dataset.keys())
output_size = 10
hidden_layer_size = 100
n_epochs = 100000

weights_initializer = keras.initializers.GlorotUniform()
#weights_initializer = tf.random_normal_initializer(mean=0.0, stddev=0.05, seed=None)

#A function that trains and validates the model and returns the MSE (for the purpose of time-constrained comparison of BSG and Mini-SGD)
def train_val_learning(run_dir, hparams):
    model = keras.models.Sequential([
            #Layer to be used as an entry point into a Network
            keras.layers.InputLayer(input_shape=[len(train_dataset.keys())]),
            #Dense layer 1
            keras.layers.Dense(hidden_layer_size, activation='sigmoid', 
                               kernel_initializer = weights_initializer,
                               name='Layer_1'),
        
            #Batch Layer
            keras.layers.BatchNormalization(),
        
            #Dense layer 2
            keras.layers.Dense(hidden_layer_size, activation='sigmoid', 
                               kernel_initializer = weights_initializer,
                               name='Layer_2'),
        
            #Batch Layer
            keras.layers.BatchNormalization(),
        
            #activation function is linear since we are doing regression
            keras.layers.Dense(output_size, activation='linear', name='Output_layer')
                                ])
    
    #Use the Mini-SGD optimizer with a dec
    optimizer = tf.keras.optimizers.SGD(learning_rate=hparams['learning_rate'], momentum=0.0,
                                        nesterov=False)
    
    #Compiling the model
    model.compile(optimizer=optimizer, 
                  loss='mean_squared_error', #Computes the mean of squares of errors between labels and predictions
                  metrics=['mean_squared_error']) #Computes the mean squared error between y_true and y_pred
    
    #Training the network
    history = model.fit(normed_train_data, train_labels, 
         epochs=hparams['epochs'],
         batch_size=len(normed_train_data), 
         verbose=1,
         #validation_split=0.2,
         callbacks=[tf.keras.callbacks.TensorBoard(run_dir + "/Heston", update_freq='batch')])
    
    return history

train_val_learning("logs/learning_sched", {'epochs': 1000, 'learning_rate': learning_rate_fn})
train_val_learning("logs/learning001", {'epochs': 1000, 'learning_rate': 0.001})

Epoch 1/1000
1/1 [==============================] - 0s 2ms/step - loss: 1.8426 - mean_squared_error: 1.8426
Epoch 2/1000
1/1 [==============================] - 0s 2ms/step - loss: 2.5115 - mean_squared_error: 2.5115
Epoch 3/1000
1/1 [==============================] - 0s 2ms/step - loss: 1.0505 - mean_squared_error: 1.0505
Epoch 4/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.3858 - mean_squared_error: 0.3858
Epoch 5/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.2213 - mean_squared_error: 0.2213
Epoch 6/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.1549 - mean_squared_error: 0.1549
Epoch 7/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.1196 - mean_squared_error: 0.1196
Epoch 8/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.0965 - mean_squared_error: 0.0965
Epoch 9/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.0804 - mean_squared_error: 0.0804
Epoch 10/1000
1/1 [=========

1/1 [==============================] - 0s 2ms/step - loss: 0.0098 - mean_squared_error: 0.0098
Epoch 77/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.0097 - mean_squared_error: 0.0097
Epoch 78/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.0096 - mean_squared_error: 0.0096
Epoch 79/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.0095 - mean_squared_error: 0.0095
Epoch 80/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.0094 - mean_squared_error: 0.0094
Epoch 81/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.0094 - mean_squared_error: 0.0094
Epoch 82/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.0093 - mean_squared_error: 0.0093
Epoch 83/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.0092 - mean_squared_error: 0.0092
Epoch 84/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.0092 - mean_squared_error: 0.0092
Epoch 85/1000
1/1 [==============

1/1 [==============================] - 0s 2ms/step - loss: 0.0061 - mean_squared_error: 0.0061
Epoch 151/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.0061 - mean_squared_error: 0.0061
Epoch 152/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.0061 - mean_squared_error: 0.0061
Epoch 153/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.0060 - mean_squared_error: 0.0060
Epoch 154/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.0060 - mean_squared_error: 0.0060
Epoch 155/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.0060 - mean_squared_error: 0.0060
Epoch 156/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.0059 - mean_squared_error: 0.0059
Epoch 157/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.0059 - mean_squared_error: 0.0059
Epoch 158/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.0059 - mean_squared_error: 0.0059
Epoch 159/1000
1/1 [=====

1/1 [==============================] - 0s 2ms/step - loss: 0.0045 - mean_squared_error: 0.0045
Epoch 225/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.0045 - mean_squared_error: 0.0045
Epoch 226/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.0045 - mean_squared_error: 0.0045
Epoch 227/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.0045 - mean_squared_error: 0.0045
Epoch 228/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.0045 - mean_squared_error: 0.0045
Epoch 229/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.0044 - mean_squared_error: 0.0044
Epoch 230/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.0044 - mean_squared_error: 0.0044
Epoch 231/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.0044 - mean_squared_error: 0.0044
Epoch 232/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.0044 - mean_squared_error: 0.0044
Epoch 233/1000
1/1 [=====

1/1 [==============================] - 0s 2ms/step - loss: 0.0036 - mean_squared_error: 0.0036
Epoch 299/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.0036 - mean_squared_error: 0.0036
Epoch 300/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.0036 - mean_squared_error: 0.0036
Epoch 301/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.0036 - mean_squared_error: 0.0036
Epoch 302/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.0036 - mean_squared_error: 0.0036
Epoch 303/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.0036 - mean_squared_error: 0.0036
Epoch 304/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.0036 - mean_squared_error: 0.0036
Epoch 305/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.0036 - mean_squared_error: 0.0036
Epoch 306/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.0036 - mean_squared_error: 0.0036
Epoch 307/1000
1/1 [=====

...

1/1 [==============================] - 0s 2ms/step - loss: 0.1353 - mean_squared_error: 0.1353
Epoch 633/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.1351 - mean_squared_error: 0.1351
Epoch 634/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.1349 - mean_squared_error: 0.1349
Epoch 635/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.1347 - mean_squared_error: 0.1347
Epoch 636/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.1345 - mean_squared_error: 0.1345
Epoch 637/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.1344 - mean_squared_error: 0.1344
Epoch 638/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.1342 - mean_squared_error: 0.1342
Epoch 639/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.1340 - mean_squared_error: 0.1340
Epoch 640/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.1339 - mean_squared_error: 0.1339
Epoch 641/1000
1/1 [=====

1/1 [==============================] - 0s 2ms/step - loss: 0.1228 - mean_squared_error: 0.1228
Epoch 707/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.1227 - mean_squared_error: 0.1227
Epoch 708/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.1225 - mean_squared_error: 0.1225
Epoch 709/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.1224 - mean_squared_error: 0.1224
Epoch 710/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.1222 - mean_squared_error: 0.1222
Epoch 711/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.1221 - mean_squared_error: 0.1221
Epoch 712/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.1219 - mean_squared_error: 0.1219
Epoch 713/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.1217 - mean_squared_error: 0.1217
Epoch 714/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.1216 - mean_squared_error: 0.1216
Epoch 715/1000
1/1 [=====

1/1 [==============================] - 0s 2ms/step - loss: 0.1117 - mean_squared_error: 0.1117
Epoch 781/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.1115 - mean_squared_error: 0.1115
Epoch 782/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.1114 - mean_squared_error: 0.1114
Epoch 783/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.1112 - mean_squared_error: 0.1112
Epoch 784/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.1111 - mean_squared_error: 0.1111
Epoch 785/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.1110 - mean_squared_error: 0.1110
Epoch 786/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.1108 - mean_squared_error: 0.1108
Epoch 787/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.1107 - mean_squared_error: 0.1107
Epoch 788/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.1105 - mean_squared_error: 0.1105
Epoch 789/1000
1/1 [=====

1/1 [==============================] - 0s 3ms/step - loss: 0.1016 - mean_squared_error: 0.1016
Epoch 855/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.1015 - mean_squared_error: 0.1015
Epoch 856/1000
1/1 [==============================] - 0s 5ms/step - loss: 0.1013 - mean_squared_error: 0.1013
Epoch 857/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.1012 - mean_squared_error: 0.1012
Epoch 858/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.1011 - mean_squared_error: 0.1011
Epoch 859/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.1010 - mean_squared_error: 0.1010
Epoch 860/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.1008 - mean_squared_error: 0.1008
Epoch 861/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.1007 - mean_squared_error: 0.1007
Epoch 862/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.1006 - mean_squared_error: 0.1006
Epoch 863/1000
1/1 [=====

1/1 [==============================] - 0s 2ms/step - loss: 0.0925 - mean_squared_error: 0.0925
Epoch 929/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.0924 - mean_squared_error: 0.0924
Epoch 930/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.0923 - mean_squared_error: 0.0923
Epoch 931/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.0922 - mean_squared_error: 0.0922
Epoch 932/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.0921 - mean_squared_error: 0.0921
Epoch 933/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.0919 - mean_squared_error: 0.0919
Epoch 934/1000
1/1 [==============================] - 0s 5ms/step - loss: 0.0918 - mean_squared_error: 0.0918
Epoch 935/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.0917 - mean_squared_error: 0.0917
Epoch 936/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.0916 - mean_squared_error: 0.0916
Epoch 937/1000
1/1 [=====

In [14]:
%tensorboard --logdir_spec=LR_sch:logs/learning_sched,LR_0.001:logs/learning001

Reusing TensorBoard on port 6012 (pid 20898), started 0:16:47 ago. (Use '!kill 20898' to kill it.)